In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [27]:
!pip install kagglehub
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mishra5001/credit-card")

print("Path to dataset files:", path)

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Path to dataset files: /Users/shehab/.cache/kagglehub/datasets/mishra5001/credit-card/versions/1


In [28]:
app_df = pd.read_csv(path+"/application_data.csv")
app_df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
app_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB


In [30]:
prev_app_df = pd.read_csv(path+"/previous_application.csv")
prev_app_df.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
merged_df = app_df.merge(prev_app_df, on='SK_ID_CURR', how='inner')
for col in merged_df.columns:
    if merged_df[col].dtype in ['float64', 'object']:
        if merged_df[col].dropna().apply(lambda x: str(x).replace('.', '', 1).isdigit()).all():
            merged_df[col] = merged_df[col].astype(int, errors='ignore')
merged_df.info()
merged_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1413701 entries, 0 to 1413700
Columns: 158 entries, SK_ID_CURR to NFLAG_INSURED_ON_APPROVAL
dtypes: float64(75), int64(51), object(32)
memory usage: 1.7+ GB


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE_x,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY_x,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,100002,1,Cash loans,M,N,Y,0,202500,406597,24700.5,...,Auto technology,24.0,low_normal,POS other with interest,365243.0,-565.0,125.0,-25.0,-17.0,0.0
1,100003,0,Cash loans,F,N,N,0,270000,1293502,35698.5,...,XNA,12.0,low_normal,Cash X-Sell: low,365243.0,-716.0,-386.0,-536.0,-527.0,1.0
2,100003,0,Cash loans,F,N,N,0,270000,1293502,35698.5,...,Furniture,6.0,middle,POS industry with interest,365243.0,-797.0,-647.0,-647.0,-639.0,0.0
3,100003,0,Cash loans,F,N,N,0,270000,1293502,35698.5,...,Consumer electronics,12.0,middle,POS household with interest,365243.0,-2310.0,-1980.0,-1980.0,-1976.0,1.0
4,100004,0,Revolving loans,M,Y,Y,0,67500,135000,6750.0,...,Connectivity,4.0,middle,POS mobile without interest,365243.0,-784.0,-694.0,-724.0,-714.0,0.0


In [32]:
merged_df.dtypes

SK_ID_CURR                     int64
TARGET                         int64
NAME_CONTRACT_TYPE_x          object
CODE_GENDER                   object
FLAG_OWN_CAR                  object
                              ...   
DAYS_FIRST_DUE               float64
DAYS_LAST_DUE_1ST_VERSION    float64
DAYS_LAST_DUE                float64
DAYS_TERMINATION             float64
NFLAG_INSURED_ON_APPROVAL    float64
Length: 158, dtype: object

In [33]:
transactions = merged_df[['SK_ID_CURR', 'SK_ID_PREV']].copy()
transactions = transactions.rename(columns={'SK_ID_PREV': 'SK_ID_RECEIVER'})

In [34]:
class UnionFind:
    def __init__(self, elements):
        self.parent = {element: element for element in elements}
        self.rank = {element: 0 for element in elements}

    def find(self, element):
        if self.parent[element] != element:
            self.parent[element] = self.find(self.parent[element]) # Path compression
        return self.parent[element]

    def union(self, element1, element2):
        root1 = self.find(element1)
        root2 = self.find(element2)

        if root1 != root2:
            if self.rank[root1] < self.rank[root2]:
                self.parent[root1] = root2
            elif self.rank[root1] > self.rank[root2]:
                self.parent[root2] = root1
            else:
                self.parent[root2] = root1
                self.rank[root1] += 1

In [35]:
unique_accounts = pd.concat([transactions['SK_ID_CURR'], transactions['SK_ID_RECEIVER']]).unique()
uf = UnionFind(unique_accounts)

for index, row in transactions.iterrows():
    uf.union(row['SK_ID_CURR'], row['SK_ID_RECEIVER'])

In [36]:
account_community_map = {account: uf.find(account) for account in unique_accounts}
merged_df['community_id'] = merged_df['SK_ID_CURR'].map(account_community_map)

In [ ]:
#merged_df.to_csv('merged_credit_card_data.csv', index=False)
#print("merged_credit_card_data.csv has been saved.")

merged_credit_card_data.csv has been saved.


In [38]:
merged_df.T

,0,1,2,3,4,5,6,7,8,9,...,1413691,1413692,1413693,1413694,1413695,1413696,1413697,1413698,1413699,1413700
SK_ID_CURR,100002,100003,100003,100003,100004,100006,100006,100006,100006,100006,...,456254,456254,456255,456255,456255,456255,456255,456255,456255,456255
TARGET,1,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
NAME_CONTRACT_TYPE_x,Cash loans,Cash loans,Cash loans,Cash loans,Revolving loans,Cash loans,Cash loans,Cash loans,Cash loans,Cash loans,...,Cash loans,Cash loans,Cash loans,Cash loans,Cash loans,Cash loans,Cash loans,Cash loans,Cash loans,Cash loans
CODE_GENDER,M,F,F,F,M,F,F,F,F,F,...,F,F,F,F,F,F,F,F,F,F
FLAG_OWN_CAR,N,N,N,N,Y,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DAYS_LAST_DUE_1ST_VERSION,125.0,-386.0,-647.0,-1980.0,-694.0,1259.0,NaN,49.0,365243.0,NaN,...,203.0,99.0,-630.0,-95.0,28.0,72.0,NaN,NaN,294.0,-319.0
DAYS_LAST_DUE,-25.0,-536.0,-647.0,-1980.0,-724.0,-151.0,NaN,365243.0,365243.0,NaN,...,365243.0,365243.0,-690.0,-215.0,-212.0,-558.0,NaN,NaN,-66.0,-409.0
DAYS_TERMINATION,-17.0,-527.0,-639.0,-1976.0,-714.0,-143.0,NaN,365243.0,365243.0,NaN,...,365243.0,365243.0,-687.0,-211.0,-204.0,-528.0,NaN,NaN,-64.0,-400.0
NFLAG_INSURED_ON_APPROVAL,0.0,1.0,0.0,1.0,0.0,0.0,NaN,0.0,0.0,NaN,...,1.0,0.0,0.0,0.0,1.0,1.0,NaN,NaN,0.0,0.0


In [39]:
merged_df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE_x,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY_x,...,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,community_id
0,100002,1,Cash loans,M,N,Y,0,202500,406597,24700.5,...,24.0,low_normal,POS other with interest,365243.0,-565.0,125.0,-25.0,-17.0,0.0,100002
1,100003,0,Cash loans,F,N,N,0,270000,1293502,35698.5,...,12.0,low_normal,Cash X-Sell: low,365243.0,-716.0,-386.0,-536.0,-527.0,1.0,100003
2,100003,0,Cash loans,F,N,N,0,270000,1293502,35698.5,...,6.0,middle,POS industry with interest,365243.0,-797.0,-647.0,-647.0,-639.0,0.0,100003
3,100003,0,Cash loans,F,N,N,0,270000,1293502,35698.5,...,12.0,middle,POS household with interest,365243.0,-2310.0,-1980.0,-1980.0,-1976.0,1.0,100003
4,100004,0,Revolving loans,M,Y,Y,0,67500,135000,6750.0,...,4.0,middle,POS mobile without interest,365243.0,-784.0,-694.0,-724.0,-714.0,0.0,100004


In [40]:
merged_df.isnull().sum()

SK_ID_CURR                        0
TARGET                            0
NAME_CONTRACT_TYPE_x              0
CODE_GENDER                       0
FLAG_OWN_CAR                      0
                              ...  
DAYS_LAST_DUE_1ST_VERSION    561106
DAYS_LAST_DUE                561106
DAYS_TERMINATION             561106
NFLAG_INSURED_ON_APPROVAL    561106
community_id                      0
Length: 159, dtype: int64

In [41]:
null_summary = (
    merged_df.isnull()
    .sum()
    .reset_index()
    .rename(columns={'index': 'Feature', 0: 'Null_Count'})
)

null_summary['Null_%'] = (null_summary['Null_Count'] / len(merged_df) * 100).round(2)
null_summary = null_summary.sort_values(by='Null_Count', ascending=False)

print(null_summary)

                         Feature  Null_Count  Null_%
134        RATE_INTEREST_PRIMARY     1408910   99.66
135     RATE_INTEREST_PRIVILEGED     1408910   99.66
48                COMMONAREA_AVG      985199   69.69
76               COMMONAREA_MEDI      985199   69.69
62               COMMONAREA_MODE      985199   69.69
..                           ...         ...     ...
36   LIVE_REGION_NOT_WORK_REGION           0    0.00
35    REG_REGION_NOT_WORK_REGION           0    0.00
34    REG_REGION_NOT_LIVE_REGION           0    0.00
33     HOUR_APPR_PROCESS_START_x           0    0.00
158                 community_id           0    0.00

[159 rows x 3 columns]


In [42]:
def drop_high_null_features(merged_df, threshold=0.4):
    """
    Drops all columns with more than 'threshold' proportion of null values.
    
    Parameters:
        df (pd.DataFrame): Input DataFrame
        threshold (float): Fraction of null values above which columns are dropped (default = 0.4)
        
    Returns:
        pd.DataFrame: Cleaned DataFrame with columns removed
    """
    # Calculate the fraction of nulls per column
    null_fraction = merged_df.isnull().mean()
    
    # Columns to drop
    cols_to_drop = null_fraction[null_fraction > threshold].index
    
    print(f"🧹 Dropping {len(cols_to_drop)} columns with more than {threshold*100}% null values")
    
    # Drop and return
    df_cleaned = merged_df.drop(columns=cols_to_drop)
    return df_cleaned

In [ ]:
merged_df_clean = drop_high_null_features(merged_df, threshold=0.4)
merged_df_clean.shape

🧹 Dropping 54 columns with more than 40.0% null values


(1413701, 105)

In [51]:
merged_df_clean.head(5)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE_x,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY_x,...,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,community_id
0,100002,1,Cash loans,M,N,Y,0,202500,406597,24700.5,...,24.0,low_normal,POS other with interest,365243.0,-565.0,125.0,-25.0,-17.0,0.0,100002
1,100003,0,Cash loans,F,N,N,0,270000,1293502,35698.5,...,12.0,low_normal,Cash X-Sell: low,365243.0,-716.0,-386.0,-536.0,-527.0,1.0,100003
2,100003,0,Cash loans,F,N,N,0,270000,1293502,35698.5,...,6.0,middle,POS industry with interest,365243.0,-797.0,-647.0,-647.0,-639.0,0.0,100003
3,100003,0,Cash loans,F,N,N,0,270000,1293502,35698.5,...,12.0,middle,POS household with interest,365243.0,-2310.0,-1980.0,-1980.0,-1976.0,1.0,100003
4,100004,0,Revolving loans,M,Y,Y,0,67500,135000,6750.0,...,4.0,middle,POS mobile without interest,365243.0,-784.0,-694.0,-724.0,-714.0,0.0,100004


In [47]:
merged_df_clean.isnull().sum()


SK_ID_CURR                        0
TARGET                            0
NAME_CONTRACT_TYPE_x              0
CODE_GENDER                       0
FLAG_OWN_CAR                      0
                              ...  
DAYS_LAST_DUE_1ST_VERSION    561106
DAYS_LAST_DUE                561106
DAYS_TERMINATION             561106
NFLAG_INSURED_ON_APPROVAL    561106
community_id                      0
Length: 105, dtype: int64

In [49]:
merged_df_clean.columns

Index(['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE_x', 'CODE_GENDER',
       'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT_x', 'AMT_ANNUITY_x',
       ...
       'CNT_PAYMENT', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION',
       'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION',
       'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL',
       'community_id'],
      dtype='object', length=105)

In [50]:
output_path = "remaining_columns.txt"

with open(output_path, "w") as f:
    for col in merged_df_clean.columns:
        f.write(col + "\n")

print(f"✅ Saved {len(merged_df_clean.columns)} column names to '{output_path}'")

✅ Saved 105 column names to 'remaining_columns.txt'


In [53]:
def display_unique_value_counts(df, sort_by_count=False):
    """
    Displays the count of unique values for each column in a DataFrame.

    Parameters:
        df (pd.DataFrame): The DataFrame to analyze
        sort_by_count (bool): If True, sorts by number of unique values (descending)

    Returns:
        pd.DataFrame: A summary table of column names and their unique counts
    """
    unique_counts = df.nunique().reset_index()
    unique_counts.columns = ['Column', 'Unique_Values']

    if sort_by_count:
        unique_counts = unique_counts.sort_values(by='Unique_Values', ascending=False)

    print(f"📊 Total columns analyzed: {len(df.columns)}")
    display(unique_counts)  # works well in Jupyter/Colab

    return unique_counts

unique_summary = display_unique_value_counts(merged_df_clean, sort_by_count=True)

📊 Total columns analyzed: 105


,Column,Unique_Values
73,SK_ID_PREV,1413701
75,AMT_ANNUITY_y,329537
0,SK_ID_CURR,291057
104,community_id,291057
40,EXT_SOURCE_2,117215
...,...,...
5,FLAG_OWN_REALTY,2
53,FLAG_DOCUMENT_8,2
52,FLAG_DOCUMENT_7,2
21,FLAG_MOBIL,1


In [58]:
columns_description = {
    "SK_ID_CURR": "ID of loan in our sample",
    "TARGET": "Target variable (1 - client with payment difficulties; 0 - all other cases)",
    "NAME_CONTRACT_TYPE_x": "Identification if loan is cash or revolving (current application)",
    "CODE_GENDER": "Gender of the client",
    "FLAG_OWN_CAR": "Flag if the client owns a car",
    "FLAG_OWN_REALTY": "Flag if client owns a house or flat",
    "CNT_CHILDREN": "Number of children the client has",
    "AMT_INCOME_TOTAL": "Income of the client",
    "AMT_CREDIT_x": "Credit amount of the loan (current application)",
    "AMT_ANNUITY_x": "Loan annuity (current application)",
    "AMT_GOODS_PRICE_x": "Price of goods for which the loan is given (current application)",
    "NAME_TYPE_SUITE_x": "Who was accompanying client when applying for the current loan",
    "NAME_INCOME_TYPE": "Client’s income type (businessman, working, maternity leave, etc.)",
    "NAME_EDUCATION_TYPE": "Level of highest education the client achieved",
    "NAME_FAMILY_STATUS": "Family status of the client",
    "NAME_HOUSING_TYPE": "Housing situation of the client (renting, living with parents, etc.)",
    "REGION_POPULATION_RELATIVE": "Normalized population of region where client lives (higher = more populated)",
    "DAYS_BIRTH": "Client's age in days at the time of application",
    "DAYS_EMPLOYED": "Days before application the person started current employment",
    "DAYS_REGISTRATION": "Days before application client changed registration",
    "DAYS_ID_PUBLISH": "Days before application client changed the identity document used for the loan",
    "FLAG_MOBIL": "Did client provide mobile phone (1=YES, 0=NO)",
    "FLAG_EMP_PHONE": "Did client provide work phone (1=YES, 0=NO)",
    "FLAG_WORK_PHONE": "Did client provide home phone (1=YES, 0=NO)",
    "FLAG_CONT_MOBILE": "Was mobile phone reachable (1=YES, 0=NO)",
    "FLAG_PHONE": "Did client provide home phone (1=YES, 0=NO)",
    "FLAG_EMAIL": "Did client provide email (1=YES, 0=NO)",
    "OCCUPATION_TYPE": "Kind of occupation the client has",
    "CNT_FAM_MEMBERS": "How many family members the client has",
    "REGION_RATING_CLIENT": "Rating of the region where client lives (1,2,3)",
    "REGION_RATING_CLIENT_W_CITY": "Rating of the region where client lives with city taken into account (1,2,3)",
    "WEEKDAY_APPR_PROCESS_START_x": "Day of the week when client applied for current loan",
    "HOUR_APPR_PROCESS_START_x": "Hour when client applied for current loan (rounded)",
    "REG_REGION_NOT_LIVE_REGION": "Flag if permanent address ≠ contact address (region level)",
    "REG_REGION_NOT_WORK_REGION": "Flag if permanent address ≠ work address (region level)",
    "LIVE_REGION_NOT_WORK_REGION": "Flag if contact address ≠ work address (region level)",
    "REG_CITY_NOT_LIVE_CITY": "Flag if permanent address ≠ contact address (city level)",
    "REG_CITY_NOT_WORK_CITY": "Flag if permanent address ≠ work address (city level)",
    "LIVE_CITY_NOT_WORK_CITY": "Flag if contact address ≠ work address (city level)",
    "ORGANIZATION_TYPE": "Type of organization where client works",
    "EXT_SOURCE_2": "Normalized score from external data source",
    "EXT_SOURCE_3": "Normalized score from external data source",
    "OBS_30_CNT_SOCIAL_CIRCLE": "Number of observations of client's social circle with 30 DPD (days past due) default",
    "DEF_30_CNT_SOCIAL_CIRCLE": "Number of client’s social circle defaults on 30 DPD",
    "OBS_60_CNT_SOCIAL_CIRCLE": "Number of observations of client's social circle with 60 DPD default",
    "DEF_60_CNT_SOCIAL_CIRCLE": "Number of client’s social circle defaults on 60 DPD",
    "DAYS_LAST_PHONE_CHANGE": "Days before application client changed phone",
    "FLAG_DOCUMENT_2": "Did client provide document 2",
    "FLAG_DOCUMENT_3": "Did client provide document 3",
    "FLAG_DOCUMENT_4": "Did client provide document 4",
    "FLAG_DOCUMENT_5": "Did client provide document 5",
    "FLAG_DOCUMENT_6": "Did client provide document 6",
    "FLAG_DOCUMENT_7": "Did client provide document 7",
    "FLAG_DOCUMENT_8": "Did client provide document 8",
    "FLAG_DOCUMENT_9": "Did client provide document 9",
    "FLAG_DOCUMENT_10": "Did client provide document 10",
    "FLAG_DOCUMENT_11": "Did client provide document 11",
    "FLAG_DOCUMENT_12": "Did client provide document 12",
    "FLAG_DOCUMENT_13": "Did client provide document 13",
    "FLAG_DOCUMENT_14": "Did client provide document 14",
    "FLAG_DOCUMENT_15": "Did client provide document 15",
    "FLAG_DOCUMENT_16": "Did client provide document 16",
    "FLAG_DOCUMENT_17": "Did client provide document 17",
    "FLAG_DOCUMENT_18": "Did client provide document 18",
    "FLAG_DOCUMENT_19": "Did client provide document 19",
    "FLAG_DOCUMENT_20": "Did client provide document 20",
    "FLAG_DOCUMENT_21": "Did client provide document 21",
    "AMT_REQ_CREDIT_BUREAU_HOUR": "Number of credit bureau enquiries one hour before application",
    "AMT_REQ_CREDIT_BUREAU_DAY": "Number of credit bureau enquiries one day before application (excluding last hour)",
    "AMT_REQ_CREDIT_BUREAU_WEEK": "Number of credit bureau enquiries one week before application (excluding last day)",
    "AMT_REQ_CREDIT_BUREAU_MON": "Number of credit bureau enquiries one month before application (excluding last week)",
    "AMT_REQ_CREDIT_BUREAU_QRT": "Number of credit bureau enquiries three months before application (excluding last month)",
    "AMT_REQ_CREDIT_BUREAU_YEAR": "Number of credit bureau enquiries one year before application (excluding last three months)",
    "SK_ID_PREV": "ID of previous credit related to loan in our sample",
    "NAME_CONTRACT_TYPE_y": "Contract product type (Cash loan, consumer loan, etc.) of previous application",
    "AMT_ANNUITY_y": "Annuity of previous application",
    "AMT_APPLICATION": "Amount of credit requested on previous application",
    "AMT_CREDIT_y": "Final credit amount granted on previous application",
    "AMT_GOODS_PRICE_y": "Goods price in previous application",
    "WEEKDAY_APPR_PROCESS_START_y": "Day of the week of previous application",
    "HOUR_APPR_PROCESS_START_y": "Hour of previous application (rounded)",
    "FLAG_LAST_APPL_PER_CONTRACT": "Flag if it was last application for the previous contract",
    "NFLAG_LAST_APPL_IN_DAY": "Flag if the application was the last of the day for the client",
    "NAME_CASH_LOAN_PURPOSE": "Purpose of the cash loan in previous application",
    "NAME_CONTRACT_STATUS": "Contract status (approved, cancelled, etc.) of previous application",
    "DAYS_DECISION": "Days relative to current application when previous application decision was made",
    "NAME_PAYMENT_TYPE": "Payment method chosen for previous application",
    "CODE_REJECT_REASON": "Reason why the previous application was rejected",
    "NAME_CLIENT_TYPE": "Old or new client flag for previous application",
    "NAME_GOODS_CATEGORY": "Type of goods applied for in previous application",
    "NAME_PORTFOLIO": "Portfolio type of previous application (CASH, POS, CAR, etc.)",
    "NAME_PRODUCT_TYPE": "Product type (x-sell or walk-in) of previous application",
    "CHANNEL_TYPE": "Channel through which client applied for previous application",
    "SELLERPLACE_AREA": "Selling area of seller place for previous application",
    "NAME_SELLER_INDUSTRY": "Industry of the seller in previous application",
    "CNT_PAYMENT": "Term of previous credit at application time",
    "NAME_YIELD_GROUP": "Grouped interest rate into small, medium, and high for previous application",
    "PRODUCT_COMBINATION": "Detailed product combination of previous application",
    "DAYS_FIRST_DRAWING": "Days relative to current application when first disbursement occurred (previous application)",
    "DAYS_FIRST_DUE": "Days relative to current application when first due date was scheduled (previous application)",
    "DAYS_LAST_DUE_1ST_VERSION": "Days relative to current application when first due date occurred (previous application)",
    "DAYS_LAST_DUE": "Days relative to current application when last due date occurred (previous application)",
    "DAYS_TERMINATION": "Days relative to current application when expected termination occurred (previous application)",
    "NFLAG_INSURED_ON_APPROVAL": "Flag if client requested insurance during previous application",
    "community_id": "Synthetic or external community identifier used for grouping or linking records"
}

In [63]:
def find_non_numeric_binaries(df: pd.DataFrame, exclude_bools: bool = True):
    """
    Find columns with exactly two unique non-null values where those values
    are NOT equivalent to {0,1}. Useful to spot string-binary fields that
    need categorical encoding.

    Parameters
    ----------
    df : pd.DataFrame
        Input dataframe.
    exclude_bools : bool, default True
        If True, columns with boolean {True, False} are treated as binary 0/1 and excluded.

    Returns
    -------
    cols : list[str]
        List of column names that have exactly two unique non-null values and are not {0,1}.
    details : dict[str, list]
        Mapping of column -> its two unique values (as seen in the data).
    """
    cols = []
    details = {}

    for col in df.columns:
        # unique non-null values
        uniq = pd.unique(df[col].dropna())

        # must be exactly two unique values
        if len(uniq) != 2:
            continue

        # normalize values to detect 0/1 forms
        norm = set()
        for v in uniq:
            # Booleans
            if isinstance(v, (bool, np.bool_)):
                if exclude_bools:
                    norm.add(int(v))  # True->1, False->0
                else:
                    norm.add(v)
                continue

            # Numeric (ints/floats)
            if isinstance(v, (int, np.integer, float, np.floating)):
                # treat 0.0/1.0 as 0/1
                if float(v).is_integer() and int(v) in (0, 1):
                    norm.add(int(v))
                else:
                    norm.add(v)
                continue

            # Strings: check if '0' or '1'
            if isinstance(v, str):
                s = v.strip()
                if s in {"0", "1"}:
                    norm.add(int(s))
                else:
                    norm.add(s.lower())  # case-insensitive compare
                continue

            # Fallback
            norm.add(v)

        # If normalized set equals {0,1}, treat as proper binary → skip
        if norm == {0, 1}:
            continue

        # Otherwise this is a two-level non 0/1 field → flag it
        cols.append(col)
        details[col] = list(uniq)

    return cols, details

cols_to_encode, sample_values = find_non_numeric_binaries(merged_df_clean)

print("Columns with exactly two unique values that are NOT 0/1:")
print(cols_to_encode)
print("\nSample values per column:")
for c, vals in sample_values.items():
    print(f"{c}: {vals}")

Columns with exactly two unique values that are NOT 0/1:
['NAME_CONTRACT_TYPE_x', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'FLAG_LAST_APPL_PER_CONTRACT']

Sample values per column:
NAME_CONTRACT_TYPE_x: ['Cash loans', 'Revolving loans']
FLAG_OWN_CAR: ['N', 'Y']
FLAG_OWN_REALTY: ['Y', 'N']
FLAG_LAST_APPL_PER_CONTRACT: ['Y', 'N']


In [69]:
def encode_selected_binaries(df: pd.DataFrame) -> pd.DataFrame:
    """
    Encode specific binary and categorical features into numeric model-ready form.
    Returns a new DataFrame with the encoded columns.
    """
    df = df.copy()

    # 1️⃣ NAME_CONTRACT_TYPE_x → is_CashLoan
    df["is_CashLoan"] = df["NAME_CONTRACT_TYPE_x"].map({
        "Cash loans": 1,
        "Revolving loans": 0
    })

    # 2️⃣ FLAG_OWN_CAR → 1/0
    df["FLAG_OWN_CAR"] = df["FLAG_OWN_CAR"].map({
        "Y": 1,
        "N": 0
    })

    # 3️⃣ FLAG_OWN_REALTY → 1/0
    df["FLAG_OWN_REALTY"] = df["FLAG_OWN_REALTY"].map({
        "Y": 1,
        "N": 0
    })

    # 4️⃣ FLAG_LAST_APPL_PER_CONTRACT → 1/0
    df["FLAG_LAST_APPL_PER_CONTRACT"] = df["FLAG_LAST_APPL_PER_CONTRACT"].map({
        "Y": 1,
        "N": 0
    })

    # 5️⃣ CODE_GENDER → M=1, F=0, XNA=0.5
    df["CODE_GENDER"] = df["CODE_GENDER"].map({
        "M": 1,
        "F": 0,
        "XNA": 0.5
    })

    # Handle missing or unseen values (keep numeric consistency)
    df["is_CashLoan"] = df["is_CashLoan"].fillna(0)
    for col in ["FLAG_OWN_CAR", "FLAG_OWN_REALTY", "FLAG_LAST_APPL_PER_CONTRACT", "CODE_GENDER"]:
        df[col] = df[col].fillna(0)

    return df

merged_df_clean = encode_selected_binaries(merged_df_clean)
# Confirm encoding
merged_df_clean[[
    "is_CashLoan",
    "FLAG_OWN_CAR",
    "FLAG_OWN_REALTY",
    "FLAG_LAST_APPL_PER_CONTRACT",
    "CODE_GENDER"
]].head()

,is_CashLoan,FLAG_OWN_CAR,FLAG_OWN_REALTY,FLAG_LAST_APPL_PER_CONTRACT,CODE_GENDER
0,1,0,1,1,1.0
1,1,0,0,1,0.0
2,1,0,0,1,0.0
3,1,0,0,1,0.0
4,0,1,1,1,1.0
